<a href="https://colab.research.google.com/github/dsaadss/cloud_project/blob/main/Project_Salamandra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title pip install firebase
!pip install firebase

In [ ]:
# @title pip install fuzzywuzzy
!pip install fuzzywuzzy

In [ ]:
# @title pip install levenshtein
!pip install levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 35.2 MB/s eta 0:00:00


In [ ]:
# @title import library
from google.colab import files
import json
from firebase import firebase
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from collections import Counter
import nltk
from nltk.chat.util import Chat, reflections

In [ ]:
# @title display_user_tab_analysis
def display_user_tab_analysis(b):
    clear_output()
    df = pd.DataFrame(data)
    # Extract unique usernames and tabs
    users = list(set(df['User']))
    tabs = list(set(df['Tab']))

    # Create the checkbox widgets for users
    user_checkboxes = [widgets.Checkbox(value=False, description=user) for user in users]
    user_selection = widgets.VBox(user_checkboxes)

    tab_dropdown = widgets.Dropdown(
        options=tabs,
        description='Tab:',
        disabled=False,
    )

    output = widgets.Output()

    # Function to calculate average time spent on each tab for the selected users and tab
    def calculate_average_time(selected_users, tab):
        tab_times = []

        for user in selected_users:
            user_data = df[(df['User'] == user) & (df['Tab'] == tab)].copy()
            user_data['Time'] = pd.to_datetime(user_data['Time'])
            user_data = user_data.sort_values(by='Time')

            open_time = None

            for index, row in user_data.iterrows():
                description = row['Description']

                if 'opened' in description:
                    open_time = row['Time']
                elif 'closed' in description and open_time:
                    duration = row['Time'] - open_time
                    tab_times.append(duration.total_seconds() / 60)  # convert to minutes
                    open_time = None

        if tab_times:
            average_time = sum(tab_times) / len(tab_times)
            return average_time
        else:
            return None

    # Function to plot the graph
    def plot_average_time(selected_users, tab):
        average_time = calculate_average_time(selected_users, tab)

        with output:
            output.clear_output()
            if average_time is not None:
                plt.figure(figsize=(8, 6))
                plt.bar([tab], [average_time], color='skyblue', width=0.4)
                plt.xlabel('Tab')
                plt.ylabel('Average Time (minutes)')
                plt.title(f'Average Time Spent on Tab "{tab}" by Selected Users')
                plt.show()
            else:
                print(f"No sufficient data available for the selected users and tab {tab}")

    def on_user_tab_change(change):
        selected_users = [cb.description for cb in user_checkboxes if cb.value]
        tab = tab_dropdown.value
        if selected_users and tab:
            plot_average_time(selected_users, tab)

    for checkbox in user_checkboxes:
        checkbox.observe(on_user_tab_change, names='value')
    tab_dropdown.observe(on_user_tab_change, names='value')

    # Create back button
    back_button = widgets.Button(description='Back', layout=widgets.Layout(width='100px', height='40px'), style={'button_color': '#a38a4e', 'font_size': '14px', 'font_color': 'white'})
    back_button.on_click(lambda b: display_main_screen())

    # Layout the widgets
    user_box = widgets.VBox([widgets.Label('Select Users:')] + user_checkboxes, layout=widgets.Layout(margin='0px 20px 0px 0px'))
    dropdowns = widgets.VBox([tab_dropdown])
    main_layout = widgets.HBox([user_box, dropdowns, output], layout=widgets.Layout(justify_content='center', align_items='center', width='100%', height='100%'))

    display(HTML(style_2))
    display(widgets.VBox([main_layout, back_button], layout=widgets.Layout(justify_content='center', align_items='center', width='100%', height='100%', background_color='#add8e6')))

In [ ]:
# @title display_document_activity_analysis
def display_document_activity_analysis(b):
    clear_output()

    # Convert data to a pandas DataFrame
    df = pd.DataFrame(data)

    # Convert the 'Time' column to datetime format
    df['Time'] = pd.to_datetime(df['Time'])

    # Remove rows where 'Document' is NaN
    df = df.dropna(subset=['Document'])

    # Function to create unique lists based on current selections
    def get_unique_options(df, dates=None, users=None):
        filtered_df = df
        if dates:
            filtered_df = filtered_df[filtered_df['Time'].dt.date.isin(dates)]
        if users:
            filtered_df = filtered_df[filtered_df['User'].isin(users)]

        # Only include documents with open or close actions
        relevant_docs = filtered_df[
            filtered_df['Description'].isin(['Open document', 'Close document'])
        ]['Document'].unique()

        unique_dates = filtered_df['Time'].dt.date.unique()
        unique_users = filtered_df['User'].unique()
        unique_documents = [doc for doc in relevant_docs if not pd.isna(doc)]

        return unique_dates, unique_users, unique_documents

    # Initial unique lists for dates
    unique_dates, unique_users, unique_documents = get_unique_options(df)

    # Create ipywidgets for interactive filtering
    date_picker = widgets.SelectMultiple(
        options=unique_dates,
        description='Date',
        disabled=False
    )

    # User checkboxes will be created dynamically based on selected dates
    user_picker_box = widgets.VBox([widgets.Checkbox(value=False, description=user) for user in unique_users])

    # Create ipywidgets for document selection
    document_picker = widgets.SelectMultiple(
        options=unique_documents,
        description='Document',
        disabled=False
    )

    # Button to update display
    update_button = widgets.Button(description="Update Display")

    # Function to update user checkboxes
    def update_user_checkboxes(users):
        checkboxes = [widgets.Checkbox(value=False, description=user) for user in users]
        user_picker_box.children = checkboxes

    # Function to update options dynamically
    def update_options(*args):
        selected_dates = date_picker.value
        selected_users = [cb.description for cb in user_picker_box.children if cb.value]
        unique_dates, unique_users, unique_documents = get_unique_options(df, selected_dates, selected_users)

        # Update user checkboxes
        update_user_checkboxes(unique_users)

        # Update document options
        document_picker.options = unique_documents

    # Attach the update function to the date picker
    date_picker.observe(update_options, 'value')

    # Function to filter data based on widget values
    def filter_data(dates, users, documents):
        filtered_df = df
        if dates:
            filtered_df = filtered_df[filtered_df['Time'].dt.date.isin(dates)]
        if users:
            filtered_df = filtered_df[filtered_df['User'].isin(users)]
        if documents:
            filtered_df = filtered_df[filtered_df['Document'].isin(documents)]

        # Only include rows with open or close actions
        filtered_df = filtered_df[
            filtered_df['Description'].isin(['Open document', 'Close document'])
        ]

        return filtered_df

    # Function to calculate total active time for each user
    def calculate_active_time(filtered_df):
        active_times = {}
        for user in filtered_df['User'].unique():
            user_df = filtered_df[filtered_df['User'] == user]
            user_df = user_df.sort_values(by='Time')

            total_active_time = timedelta()
            open_time = None

            for _, row in user_df.iterrows():
                if row['Description'] == 'Open document':
                    open_time = row['Time']
                elif row['Description'] == 'Close document' and open_time:
                    total_active_time += (row['Time'] - open_time)
                    open_time = None

            active_times[user] = total_active_time
        return active_times

    # Function to update and display the filtered data
    def update_display(button):
        selected_dates = date_picker.value
        selected_users = [cb.description for cb in user_picker_box.children if cb.value]
        selected_documents = document_picker.value

        filtered_df = filter_data(selected_dates, selected_users, selected_documents)
        active_times = calculate_active_time(filtered_df)

        with output:
            clear_output(wait=True)
            display(filtered_df)

            # Plotting the timeline graph for filtered data
            open_close_df = filtered_df[
                filtered_df['Description'].isin(['Open document', 'Close document'])
            ]
            plt.figure(figsize=(7, 4))

            for user, group in open_close_df.groupby('User'):
                plt.plot(group['Time'], group['Description'], marker='o', linestyle='-', label=f"{user} (Active: {active_times[user]})")

            plt.xlabel('Time', fontsize=10)
            plt.ylabel('Action', fontsize=10)
            plt.title('Document Open and Close Timeline by User', fontsize=12)
            plt.legend(fontsize=8)
            plt.grid(True)
            plt.xticks(rotation=45, fontsize=8)
            plt.yticks(fontsize=8)

            # Format the x-axis to display only the hour and minute
            ax = plt.gca()
            ax.xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%H:%M'))

            plt.tight_layout()
            plt.show()

    # Create an output widget for displaying the filtered data and plot
    output = widgets.Output()

    # Attach the update display function to the button
    update_button.on_click(update_display)

    # Create back button
    back_button = widgets.Button(description='Back', layout=widgets.Layout(width='100px', height='40px'), style={'button_color': '#a38a4e', 'font_size': '14px', 'font_color': 'white'})
    back_button.on_click(lambda b: display_main_screen())

    # Display the widgets horizontally
    widget_box = widgets.VBox([widgets.HBox([date_picker, user_picker_box, document_picker]), update_button, back_button])
    display(HTML(style_2))
    display(widget_box, output)

    # Initial call to populate user checkboxes based on initial date selection
    update_options()

In [ ]:
# @title display_common_actions_and_user_activity
def display_common_actions_and_user_activity(b):
    clear_output()

    # Extract actions and timestamps
    actions = []
    user_activity = {}

    for entry in data:
        actions.append(entry["Description"])
        user = entry["User"]
        time = datetime.strptime(entry["Time"], '%Y-%m-%d %H:%M:%S')

        if user not in user_activity:
            user_activity[user] = []
        user_activity[user].append(time)

    # Find the most common actions
    common_actions = Counter(actions).most_common(6)  # Get up to 6 most common actions

    # Create user activity DataFrame
    user_df = pd.DataFrame([
        {'User': user, 'Time': time}
        for user, times in user_activity.items()
        for time in times
    ])

    # Set index to 'Time'
    user_df.set_index('Time', inplace=True)

    # Create an output widget to manage plot display
    output = widgets.Output()

    # Function to plot the most common actions
    def plot_common_actions(num_actions):
        with output:
            clear_output(wait=True)
            selected_actions = common_actions[:num_actions]
            print(f"Most common actions ({num_actions}):", selected_actions)

            # Plot the most common actions
            action_labels, action_counts = zip(*selected_actions)
            plt.figure(figsize=(10, 5))
            plt.bar(action_labels, action_counts, color='skyblue')
            plt.title(f'{num_actions} Most Common Actions')
            plt.xlabel('Actions')
            plt.ylabel('Counts')
            plt.xticks(rotation=45)
            plt.show()

    # Function to plot user activity
    def plot_user_activity(selected_user):
        with output:
            clear_output(wait=True)
            # Filter data based on the selected user
            user_filtered_df = user_df[user_df['User'] == selected_user]

            # Resample to count activities per month
            activity_counts = user_filtered_df.resample('M').size()

            # Plot user activity over time
            plt.figure(figsize=(15, 7))
            activity_counts.plot(kind='bar', color='skyblue')
            plt.title(f'User Activity Over Time for {selected_user}')
            plt.xlabel('Month')
            plt.ylabel('Activity Count')
            plt.xticks(rotation=45)
            plt.show()

    # Create a dropdown menu for selecting the number of actions
    num_actions_widget = widgets.Dropdown(
        options=list(range(1, 7)),
        value=4,
        description='Num Actions:'
    )

    # Create a dropdown menu for selecting the user
    user_widget = widgets.Dropdown(
        options=list(user_activity.keys()),
        description='Select User:'
    )

    # Button to plot the most common actions
    plot_actions_button = widgets.Button(description="Plot Actions")

    # Button to plot user activity
    plot_user_button = widgets.Button(description="Plot User Activity")

    # Define button click events
    def on_plot_actions_button_clicked(b):
        plot_common_actions(num_actions_widget.value)

    def on_plot_user_button_clicked(b):
        plot_user_activity(user_widget.value)

    # Assign button click events
    plot_actions_button.on_click(on_plot_actions_button_clicked)
    plot_user_button.on_click(on_plot_user_button_clicked)

    # Create back button
    back_button = widgets.Button(description='Back', layout=widgets.Layout(width='100px', height='40px'), style={'button_color': '#a38a4e', 'font_size': '14px', 'font_color': 'white'})
    back_button.on_click(lambda b: display_main_screen())

    # Display widgets and output
    widget_box = widgets.VBox([widgets.HBox([num_actions_widget, plot_actions_button, user_widget, plot_user_button]), back_button])
    display(HTML(style_2))
    display(widget_box, output)

In [ ]:
# @title display_document_time_analysis
def display_document_time_analysis(b):
  clear_output()
  # Helper function to parse datetime
  def parse_time(time_str):
     return datetime.strptime(time_str, "%Y-%m-%d %H:%M:%S")

  # Process the data
  user_times = {}
  current_open = {}

  for entry in data:
      user = entry['User']
      document = entry['Document']
      description = entry['Description']
      timestamp = parse_time(entry['Time'])

      if description == "Close document":
          current_open[(user, document)] = timestamp
      elif description == "Open document" and (user, document) in current_open:
          end_time = current_open.pop((user, document))
          time_diff = (end_time - timestamp ).total_seconds()/3600
          if document not in user_times:
              user_times[document] = {}
          if user not in user_times[document]:
              user_times[document][user] = 0
          user_times[document][user] += time_diff


#  Define a list of colors for the graphs
  colors = ['blue', 'green', 'red', 'cyan', 'magenta', 'yellow']

  # Output widget to display the plots
  output = widgets.Output()
  with output:
        # Plot the data
    for i, (document, users) in enumerate(user_times.items()):
      users = list(users.items())
      users.sort(key=lambda x: x[1], reverse=True)
      usernames, hours = zip(*users)

    # Ensure there are more colors available than the number of documents
      color = colors[i % len(colors)]

      plt.figure(figsize=(8, 4))

      bars = plt.bar(usernames, hours, color=color, width=0.4)
      for bar in bars:
        height = bar.get_height()
        if height > 1:
          plt.annotate(f'{height:.2f} hours',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    ha='center', va='bottom')
        elif (height < 1):
          plt.annotate(f'{height:.2f} hours /{60*height:.2f} seconds',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    ha='center', va='bottom')

      # plt.xlabel('Users')
      plt.ylabel('Hours Spent')
      plt.title(f'Time Spent on document: {document}')
      plt.tight_layout()
      plt.show()



# Create back button
  back_button = widgets.Button(description='Back', layout=widgets.Layout(width='100px', height='40px'), style={'button_color': '#a38a4e', 'font_size': '14px', 'font_color': 'white'})
  back_button.on_click(lambda b: display_main_screen())

    # Display the plots and back button
  display(HTML(style_2))
  display(widgets.VBox([output, back_button], layout=widgets.Layout(justify_content='center', align_items='center', width='100%', height='100%', background_color='#add8e6')))

In [ ]:
# @title analyze_document_user_activity
def analyze_document_user_activity(b=None):
    clear_output()


    # Convert the data to a DataFrame
    df = pd.DataFrame(data)

    # Convert the 'Time' column to datetime format
    df['Time'] = pd.to_datetime(df['Time'])

    # Create a new column for the date only
    df['Date'] = df['Time'].dt.date

    # Get the list of unique documents and users
    documents = df['Document'].unique()
    users = list(df['User'].unique()) + ['All Users']  # Add 'All Users' option

    # Create dropdown widgets
    document_dropdown = widgets.Dropdown(
        options=documents,
        description='Document:',
        value=documents[0]
    )

    user_dropdown = widgets.Dropdown(
        options=users,
        description='User:',
        value='All Users'
    )

    # Create an output widget to hold the graph
    output = widgets.Output()

    # Function to update the graph
    def update_graph(change):
        with output:
            # Clear the output widget
            clear_output(wait=True)

            document = document_dropdown.value
            user = user_dropdown.value

            # Filter the DataFrame based on the selected document
            filtered_df = df[df['Document'] == document]

            # Further filter based on selected user, if not 'All Users'
            if user != 'All Users':
                filtered_df = filtered_df[filtered_df['User'] == user]

            # Ensure that the Tab column exists
            if 'Tab' not in filtered_df.columns:
                print("Error: The 'Tab' column does not exist in the DataFrame.")
                return

            # Count the number of actions per Tab
            tab_activity = filtered_df.groupby(['Tab']).size().reset_index(name='ActivityCount')

            # Clear the current figure
            plt.clf()

            # Create the bar chart
            plt.bar(tab_activity['Tab'], tab_activity['ActivityCount'], color='skyblue')

            # Add labels and title
            plt.xlabel('Tab')
            plt.ylabel('Number of Activities')
            plt.title(f'Activity Count for {document} ({user})')
            plt.xticks(rotation=45, ha='right')
            plt.show()

    # Attach the update_graph function to the dropdowns' value change events
    document_dropdown.observe(update_graph, names='value')
    user_dropdown.observe(update_graph, names='value')

    # Create back button
    back_button = widgets.Button(description='Back', layout=widgets.Layout(width='100px', height='40px'), style={'button_color': '#a38a4e', 'font_size': '14px', 'font_color': 'white'})
    back_button.on_click(lambda b: display_main_screen())

    # Display the dropdown widgets, output, and back button
    display(HTML(style_2))
    display(widgets.VBox([document_dropdown, user_dropdown, output, back_button], layout=widgets.Layout(justify_content='center', align_items='center', width='100%', height='100%', background_color='#add8e6')))

    # Initial graph update
    update_graph(None)

In [ ]:
# @title plot_daily_activity_by_user
def plot_daily_activity_by_user(b):
    clear_output()


    # Convert the data to a DataFrame
    df = pd.DataFrame(data)

    # Convert the 'Time' column to datetime format
    df['Time'] = pd.to_datetime(df['Time'])

    # Create a new column for the date only
    df['Date'] = df['Time'].dt.date

    # Get the list of unique users
    users = df['User'].unique()

    # Create dropdown widget for selecting users
    user_dropdown = widgets.SelectMultiple(
        options=users,
        value=[users[0]],  # default value: first user
        description='Users',
        disabled=False
    )

    # Create an output widget to hold the graph
    output = widgets.Output()

    # Function to update the graph
    def update_graph(change):
        with output:
            # Clear the output widget
            clear_output(wait=True)

            selected_users = user_dropdown.value

            if not selected_users:
                print("No data selected")
                return

            # Filter DataFrame based on selected users
            filtered_df = df[df['User'].isin(selected_users)]

            if filtered_df.empty:
                print("No data available for the selected users")
                return

            # Aggregate activity counts by date and user
            activity_counts = filtered_df.groupby(['Date', 'User']).size().reset_index(name='ActivityCount')

            # Clear the current figure
            plt.clf()

            # Create the line plot
            plt.figure(figsize=(10, 6))
            for user in selected_users:
                user_data = activity_counts[activity_counts['User'] == user]
                plt.plot(user_data['Date'], user_data['ActivityCount'], marker='o', linestyle='-', label=user)

            # Add labels and title
            plt.xlabel('Date')
            plt.ylabel('Number of Activities')
            plt.title('Daily Activity Trend by User')
            plt.legend()
            plt.xticks(rotation=45, ha='right')
            plt.grid(True)
            plt.show()

    # Attach the update_graph function to the dropdown's value change event
    user_dropdown.observe(update_graph, names='value')

    # Create back button
    back_button = widgets.Button(description='Back', layout=widgets.Layout(width='100px', height='40px'), style={'button_color': '#a38a4e', 'font_size': '14px', 'font_color': 'white'})
    back_button.on_click(lambda b: display_main_screen())

    # Display the dropdown widget, output, and back button
    display(HTML(style_2))
    display(widgets.VBox([user_dropdown, output, back_button], layout=widgets.Layout(justify_content='center', align_items='center', width='100%', height='100%', background_color='#add8e6')))

    # Initial graph update
    update_graph(None)

In [ ]:
# @title remove_stop_words
def remove_stop_words(index):

    stop_words = {'that','you','of','n','see','type','also','all','a', 'an', 'the', 'and', 'or', 'in', 'on', 'at', 'to','context','tab','time','document','descript','1st','lab','user'}
    for stop_word in stop_words:
        if stop_word in index:
            del index[stop_word]
    return index

In [ ]:
# @title sort_index_by_values
def sort_index_by_values(index):
    return dict(sorted(index.items(), key=lambda item: item[1], reverse=True))

In [ ]:
# @title apply_stemming
from nltk.stem import PorterStemmer

def apply_stemming(index):
    stemmer = PorterStemmer()
    stemmed_index = {}
    for word, count in index.items():
        stemmed_word = stemmer.stem(word)
        if stemmed_word in stemmed_index:
            stemmed_index[stemmed_word] += count
        else:
            stemmed_index[stemmed_word] = count
    return stemmed_index

In [ ]:
# @title word_count_to_dict
import string

def word_count_to_dict(text):
    # Normalize the string
    text = text.lower()

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Split the string into words
    words = text.split()

    # Initialize the dictionary
    word_count = {}

    # Count the occurrences of each word
    for word in words:
        if word in word_count:
            word_count[word] += 1
        else:
            word_count[word] = 1

    return word_count

In [ ]:
# @title index_words
import re

def index_words(soup):
    index = {}
    words = re.findall(r'\w+', soup.get_text())
    for word in words:
        word = word.lower()
        if word in index:
            index[word] += 1
        else:
            index[word] = 1
    return index

In [ ]:
# @title fetch_page
import requests
from bs4 import BeautifulSoup

def fetch_page(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        return soup
    else:
        return None

In [ ]:
# @title get_index_word_from_on_shape
def get_index_word_from_on_shape():
        url = 'https://cad.onshape.com/help/Content/Glossary/glossary.htm?tocpath=_____19'
        soup = fetch_page(url)
        if soup is None:
                print("Failed to fetch the page.")
        else:
              print("Page fetched successfully.")

        index = index_words(soup)
        index = remove_stop_words(index)
        index = apply_stemming(index)
        return index

In [ ]:
# @title sort_index_by_values
def sort_index_by_values(index):
    return dict(sorted(index.items(), key=lambda item: item[1], reverse=True))

In [ ]:
# @title compare_json_onshape_index

def compare_json_onshape_index(b):
    on_shape_index = get_index_word_from_on_shape()
    filtered_json_string = {}

    json_string = json.dumps(data)
    json_string = word_count_to_dict(json_string)
    json_string = remove_stop_words(json_string)
    json_string = apply_stemming(json_string)

    for key, value in json_string.items():
        if key in on_shape_index:
            filtered_json_string[key] = value

    filtered_json_string = sort_index_by_values(filtered_json_string)

    clear_output()

    # Create DataFrame for filtered_json_string
    filtered_df = pd.DataFrame(filtered_json_string.items(), columns=['Term', 'Freq'])

    # Create DataFrame for on_shape_index
    on_shape_index=sort_index_by_values(on_shape_index)
    on_shape_df = pd.DataFrame(on_shape_index.items(), columns=['Term', 'Freq'])

    # Create output widgets for the tables
    filtered_output = widgets.Output()
    on_shape_output = widgets.Output()

    with filtered_output:
        display(filtered_df)

    with on_shape_output:
        display(on_shape_df)

    # Create back button
    back_button = widgets.Button(description='Back', layout=widgets.Layout(width='100px', height='40px'), style={'button_color': '#a38a4e'})
    back_button.on_click(lambda b: display_main_screen())

    # Create labels for the tables
    filtered_label = widgets.Label(value="Filtered JSON Index")
    on_shape_label = widgets.Label(value="On Shape Index")

    # Display the tables and back button
    display(widgets.VBox([
        widgets.HBox([widgets.VBox([filtered_label, filtered_output]), widgets.VBox([on_shape_label, on_shape_output])],
                     layout=widgets.Layout(justify_content='space-around', align_items='flex-start', width='100%')),
        back_button
    ]))

In [ ]:
# @title display_time_spent_per_document
def display_time_spent_per_document(b):
    clear_output()

    # Create DataFrame from data
    df = pd.DataFrame(data)

    # Convert Time column to datetime
    df['Time'] = pd.to_datetime(df['Time'])

    # Extract date part of the Time
    df['Date'] = df['Time'].dt.date

    # Sort by Document, User, Date, and Time
    df = df.sort_values(by=['Document', 'User', 'Date', 'Time'])

    # Calculate time differences within each date
    df['Time_Diff'] = df.groupby(['Document', 'User', 'Date'])['Time'].diff().shift(-1)

    # Drop NaN values (last action of each document-user-date combination)
    df = df.dropna(subset=['Time_Diff'])

    # Convert Time_Diff to hours
    df['Time_Diff'] = df['Time_Diff'].dt.total_seconds() / 3600

    # Output widget to display the plots
    output = widgets.Output()

    def plot_data(b):
        with output:
            clear_output()

            # Group by Document and User, then sum the time differences
            time_spent = df.groupby(['Document', 'User'])['Time_Diff'].sum().unstack().fillna(0)

            # Plotting
            ax = time_spent.plot(kind='bar', stacked=True, figsize=(14, 8))
            plt.title('Time Spent on Each Document by User')
            plt.xlabel('Document')
            plt.ylabel('Time Spent (hours)')

            # Annotate each bar with the value
            for container in ax.containers:
                ax.bar_label(container)

            # Show the plot
            plt.show()

    # Button to trigger the plot
    plot_button = widgets.Button(description='Plot', layout=widgets.Layout(width='100px', height='40px'))
    plot_button.on_click(plot_data)

    # Create back button
    back_button = widgets.Button(description='Back', layout=widgets.Layout(width='100px', height='40px'), style={'button_color': '#a38a4e', 'font_size': '14px', 'font_color': 'white'})
    back_button.on_click(lambda b: display_main_screen())

    # Display the plot button, output, and back button
    display(HTML("<style>.widget-label { font-size: 16px; }</style>"))
    display(widgets.VBox([plot_button, output, back_button], layout=widgets.Layout(justify_content='center', align_items='center', width='100%', height='100%', background_color='#add8e6')))

In [ ]:
# @title keywords for chat bot
keywords_for_functions = {
    display_user_tab_analysis: ["average time per tab", "average time", "tab time"],
    display_document_activity_analysis: ["timeline of document opening and closing", "document timeline", "opening closing"],
    display_common_actions_and_user_activity: ["most frequent actions and number of actions per user", "frequent actions", "actions per user"],
    display_document_time_analysis: ["user work time on project", "work time", "project time"],
    analyze_document_user_activity: ["distribution of operations by document", "operations distribution", "document operations"],
    plot_daily_activity_by_user: ["amount of activity of each user over time", "activity amount", "user activity"],
    compare_json_onshape_index: ["json index", "json", "log index", "index json"],
    display_time_spent_per_document: ["distribution - most used document", "most used document", "document distribution"]
}

In [ ]:
# @title main func
from google.colab import files
import json
from firebase import firebase
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from collections import Counter
from fuzzywuzzy import fuzz


# Global data variable
data = None

def upload_json():
    global data
    uploaded = files.upload()
    for file_name in uploaded.keys():
        print(f'Uploaded file: {file_name}')
        with open(file_name, 'r') as f:
            data = json.load(f)
        return data

def get_data_from_cloud():
    global data
    res = firebase.get('./', None)
    data = [key for key in res]
    return data

def on_button_clicked(b):
    choice = radio_buttons.value
    if choice == 'Upload JSON File (will not saving in the DB)':
        upload_json()
    elif choice == 'Use Data from Cloud':
        get_data_from_cloud()
    display_main_screen()

# CSS styling for the first screen
style_1 = """
    <style>
    .widget-label {
        font-size: 20px !important;
        color: black !important;
    }
    .centered {
        display: flex;
        justify-content: center;
        align-items: center;
        height: 100vh;
    }
    .centered > * {
        margin: 10px;
    }
    .custom-button .p-Widget.jupyter-widgets.widget-button {
        background-color: white !important;
        color: white;
        font-size: 20px !important;
        border: 2px solid #000000;
    }
    </style>
    """

# CSS styling for the second screen
style_2 = """
<style>
    .container {
        display: flex;
        justify-content: center;
        background-color: #add8e6;
        padding: 20px;
    }
    .middle-container {
        display: flex;
        flex-wrap: wrap;
        justify-content: center;
        width: 60%;
    }
    .button {
        margin: 10px;
    }
    .button > button {
        width: 300px;
        height: 50px;
        font-size: 14px;
        font-weight: bold;
        background-color: #4CAF50;
        color: black;
        border: none;
        cursor: pointer;
    }
    .button > button:hover {
        background-color: #45a049;
    }
    .custom-container {
        display: flex;
        flex-direction: column;
        background-color: #add8e6;
        padding: 20px;
    }
    .custom-container label, .custom-container .widget-label {
        color: blue;
    }
</style>
"""

# Function to create a button and link it to a function
def create_button(description, func):
    button = widgets.Button(description=description, layout=widgets.Layout(width='500px', height='50px'), style={'button_color': '#a38a4e', 'font_size': '14px', 'font_color': 'white'})
    button.on_click(func)
    button.add_class('button')
    return button

# Function to display the main screen
def display_main_screen():
    clear_output()
    display(HTML(style_2))
    display(main_container)


text_for_chat = widgets.Text(
    value='',
    placeholder='הזן הודעה...',
    description='הודעה:',
    disabled=False
       )

def on_send_button_click(b):
    entered_text = text_for_chat.value.lower().strip()
    best_match = None
    highest_ratio = 0


    for func, keywords in keywords_for_functions.items():
        for keyword in keywords:
            ratio = fuzz.partial_ratio(entered_text, keyword.lower())
            if ratio > highest_ratio:
                highest_ratio = ratio
                best_match = func

    if highest_ratio > 70:  # Threshold for considering a match
        best_match(b)
    else:
        text_for_chat.value = 'Command not recognized. Please try again.'


# Define the main function
def main():
    global radio_buttons, button, main_container

    # Create radio buttons for user choice
    radio_buttons = widgets.RadioButtons(
        options=['Upload JSON File (will not saving in the DB)', 'Use Data from Cloud'],
        description='Choose:',
        layout=widgets.Layout(width='auto')
    )

    # Create a button to submit the choice
    button = widgets.Button(description="Submit")
    button.add_class("custom-button")
    button.on_click(on_button_clicked)

    # Center the widgets
    container = widgets.VBox([radio_buttons, button], layout=widgets.Layout(align_items='center'))

    # Display the CSS and widgets
    display(HTML(style_1))
    display(widgets.VBox([container], layout=widgets.Layout(justify_content='center', align_items='center', height='300px')))

    # Buttons descriptions in Hebrew
    middle_button_descriptions = [
        ("average time per tab", display_user_tab_analysis),
        ("timeline of document opening and closing", display_document_activity_analysis),
        ("most frequent actions and number of actions per user", display_common_actions_and_user_activity),
        ("user work time on project",display_document_time_analysis),
        ("distribution of operations by document", analyze_document_user_activity),
        ("amount of activity of each user over time", plot_daily_activity_by_user),
        ("json index",compare_json_onshape_index),
        ("distribution - most used document",display_time_spent_per_document)
    ]

    # Create buttons
    middle_buttons = [create_button(description, func) for description,
func in middle_button_descriptions]
    chat_bot_widget = widgets.Button(
    description='send',
    layout=widgets.Layout(width='500px', height='50px'),
    style={'button_color': '#a38a4e', 'font_size': '14px', 'font_color': 'white'}
          )

    chat_bot_widget.on_click(lambda b: on_send_button_click(chat_bot_widget))

    # Output widget for chatbot responses
    output = widgets.Output()
    '''
    chat_bot_widget.on_click(chatbot_input_handler(text_for_chat))

    # Display chatbot UI
    display(HTML("<h2>Chatbot Interface</h2>"))
    display(input_box)
    display(send_button)
    display(output)
'''
    #on_send_button_click(chat_bot_widget,text_for_chat)

    horzien_box_for_chat = widgets.HBox([chat_bot_widget,text_for_chat])
    # Create containers to hold the buttons
    middle_container = widgets.VBox(middle_buttons)
    middle_container.layout.flex_flow = 'row wrap'
    middle_container.layout.justify_content = 'center'
    middle_container.add_class('middle-container')

    # Main container to hold the middle container
    main_container = widgets.HBox([middle_container,horzien_box_for_chat])
    main_container.add_class('container')

In [ ]:
##run this to start the program##

main()